# Sentiment Analysis using Transfer Learning - Roberta

# Step 1: Install And Import Python Libraries

In step 1, we will install and import python libraries.

Firstly, let's import `transformers` and `datasets`.

In [ ]:
# Install libraries
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00


After installing the python packages, we will import the python libraries.
* `pandas` and `numpy` are imported for data processing.
* `train_test_split` is imported from `sklearn` to split dataset.
* `tensorflow` and `transformers` are imported for modeling.
* `Dataset` is imported for the Hugging Face dataset format.
* The `accuracy_score` is imported for model performance evaluation.

In [ ]:
# Data processing
import pandas as pd
import numpy as np

# Train test split
from sklearn.model_selection import train_test_split

# Modeling
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Hugging Face Dataset
from datasets import Dataset

# Import accuracy_score to check performance
from sklearn.metrics import accuracy_score

# Step 2: Download And Read Data

Now let's read the data into a `pandas` dataframe and see what the dataset looks like.

The dataset has two columns. One column contains the reviews and the other column contains the sentiment label for the review.

In [ ]:
# Read in data
sst_train = pd.read_csv('/content/sample_data/SST_Train_Data.csv')
sst_test = pd.read_csv('/content/sample_data/SST_Test_Data.csv')
sst_dev = pd.read_csv('/content/sample_data/SST_Dev_Data.csv')

# Take a look at the data
sst_train.head()

reviews     label  label_Value
0  the rock is destined to be the 21st century s ...  positive            2
1  the gorgeously elaborate continuation of  the ...  positive            2
2  singercomposer bryan adams contributes a slew ...  positive            2
3  you  would think by now america would have had...   neutral            1
4                yet the act is still charming here   positive            2

`.info` helps us to get information about the dataset.

From the output, we can see that this data set has 8117 records and no missing data. The `reviews` column is the `object` type and the `label_Value` column is the `int64` type.

In [ ]:
# Get the dataset information
sst_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8117 entries, 0 to 8116
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   reviews      8117 non-null   object
 1   label        8117 non-null   object
 2   label_Value  8117 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 190.4+ KB


In [ ]:
# Check the label distribution
sst_train['label_Value'].value_counts()

2    3446
0    3122
1    1549
Name: label_Value, dtype: int64

# Step 3: Tokenize Text

In [ ]:
## Tokenize Reviews
# Tokenizer from a pretrained model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

train_data = sst_train["reviews"].tolist()
test_data = sst_test["reviews"].tolist()
# Tokenize the reviews
tokenized_data_train = tokenizer(train_data, return_tensors="np", padding=True)
tokenized_data_test = tokenizer(test_data, return_tensors="np", padding=True)

# Labels are one-dimensional numpy or tensorflow array of integers
labels_train = sst_train["label_Value"].tolist()
labels_test = sst_test["label_Value"].tolist()
labels_train = tf.convert_to_tensor(labels_train, dtype=tf.int32)
labels_test = tf.convert_to_tensor(labels_test, dtype=tf.int32)

# Tokenized ids
print(tokenized_data_train["input_ids"][0])

[    0   627  3152    16 19083     7    28     5   733   620  3220   579
    92  1437  2764   260  1437     8    14    37   579   164     7   146
    10 17158   190  2388    87  4709   282   279  8447  5557   329 47129
  1437  1236 12001  3998  5247  3538  9656  1794    50 11235  2987   842
  9487  1437     2     1     1     1     1     1     1     1     1     1
     1     1     1     1     1]


# Step 4: Compile Transfer Learning Model for Sentiment Analysis

In [ ]:
# Load the pre-trained model
transformer_model = TFAutoModelForSequenceClassification.from_pretrained(
   "cardiffnlp/twitter-roberta-base-sentiment-latest"
)
#model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


After loading the pretrained model, we will compile the model.
* `SparseCategoricalCrossentropy` is used as the loss function, but the Hugging Face documentation mentioned that Hugging Face models automatically choose a loss that is appropriate for their task and model architecture if the loss is not explicitly specified.
* `from_logits=True` informs the loss function that the output values are logits before applying softmax, so the values do not represent probabilities.
* We are using Adam as the optimizer and the number `5e-6` is the learning rate. A smaller learning rate corresponds to a more stable weights value update and a slower training process.
* `accuracy` is used as the metrics because we have a balanced dataset.

In [ ]:

text_X_train=sst_train["reviews"]
text_X_test=sst_test["reviews"]
# list_text_X_train = text_X_train.tolist()
# list_text_X_test = text_X_test.tolist()


In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [29]:
## Tokenize Reviews
# Tokenizer from a pretrained model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

train_data = sst_train["reviews"].tolist()
test_data = sst_test["reviews"].tolist()
# Tokenize the reviews
tokenized_data_train = tokenizer(train_data, return_tensors="np", padding=True)
tokenized_data_test = tokenizer(test_data, return_tensors="np", padding=True)

# Labels are one-dimensional numpy or tensorflow array of integers
labels_train = sst_train["label_Value"].tolist()
labels_test = sst_test["label_Value"].tolist()
labels_train = tf.convert_to_tensor(labels_train, dtype=tf.int32)
labels_test = tf.convert_to_tensor(labels_test, dtype=tf.int32)


# Tokenized ids
print(tokenized_data_train["input_ids"][1])
print(tokenized_data_train["attention_mask"][1])
type(tokenized_data_train["input_ids"][1])

[    0   627 39966  9412 10986 18719     9  1437     5 30722     9     5
 11987  1437 26105    16    98  1307    14    10  6730     9  1617    64
    45 17327  6190 12094 37124 19709   181  5906 10267  1478   579  4939
  3360     9  1236 28015     7   462   330  2495   579  1692 25581  1437
     2     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


numpy.ndarray

In [37]:
tokenized_data_train["input_ids"].shape
tokenized_data_train["attention_mask"].shape

(8117, 65)

In [ ]:
last_hidden_states = transformer_model(input_ids_array_train, attention_mask_array_train).detach()

In [ ]:
# Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile model
model.compile(optimizer=Adam(5e-6), loss=loss, metrics=['accuracy'])

# Step 5: Train Transfer Learning Model for Sentiment Analysis

In step 5, we will talk about how to fit a transfer learning model using Tensorflow Keras.

When fitting the model, we convert the tokenized dataset into a dictionary for Keras. `batch_size=4` means that four reviews are processed for each weights and bias update. `epochs=2` means that the model fitting process will go through the training dataset 2 times.

We can see that the accuracy is above 90 percent in just 2 epochs.

In [ ]:
# Fit the model
model.fit(dict(tokenized_data_train),
          labels_train,
          validation_data=(dict(tokenized_data_test), labels_test),
          batch_size=4,
          epochs=2)

Epoch 1/2
 658/2030 [========>.....................] - ETA: 1:34:24 - loss: 0.9071 - accuracy: 0.5813

# Step 6: Sentiment Analysis Transfer Learning Model Prediction and Evaluation

In [ ]:
# Predictions
y_test_predict = model.predict(dict(tokenized_data_test))['logits']

# First 5 predictions
y_test_predict[:5]

67/67 [==============================] - 12s 138ms/step


array([[-2.1745813,  4.9053063, -3.2688015],
       [-4.33564  , -1.0755202,  5.0015435],
       [-5.835117 ,  1.378774 ,  3.7665915],
       [-3.865402 , -1.8417912,  5.4970236],
       [-2.2431595, -2.1613474,  4.6802683]], dtype=float32)

To get the predicted probabilities, we need to apply softmax on the predicted logit values.

After applying softmax, we can see that the predicted probability for each review sums up to 1.

In [ ]:
# Predicted probabilities
y_test_probabilities = tf.nn.softmax(y_test_predict)

# First 5 predicted probabilities
y_test_probabilities[:5]

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[8.4092299e-04, 9.9887758e-01, 2.8154152e-04],
       [8.7877823e-05, 2.2894500e-03, 9.9762267e-01],
       [6.1922598e-05, 8.4101215e-02, 9.1583687e-01],
       [8.5828338e-05, 6.4934249e-04, 9.9926478e-01],
       [9.8243309e-04, 1.0661873e-03, 9.9795145e-01]], dtype=float32)>

To get the predicted labels, `argmax` is used to return the index of the maximum probability for each review, which corresponds to the labels of zeros and ones.

In [ ]:
# Predicted label
y_test_class_preds = np.argmax(y_test_probabilities, axis=1)

# First 5 predicted labels
y_test_class_preds[:10]

array([1, 2, 2, 2, 2, 2, 2, 1, 2, 2])

`accuracy_score` is used to evaluate the model performance. We can see that the customized sentiment analysis model with transfer learning gives us 92.5% accuracy, meaning that the predictions are correct 92.5% of the time.

# Step 7 Saving Model

In step 7, we will talk about how to save the model and reload it for prediction.

`tokenizer.save_pretrained` saves the tokenizer information to the drive and `model.save_pretrained` saves the model to the drive.

In [ ]:
# Save tokenizer
tokenizer.save_pretrained('./sentiment_transfer_learning_tensorflow/')

# Save model
model.save_pretrained('./sentiment_transfer_learning_tensorflow/')

We can load the saved tokenizer later using `AutoTokenizer.from_pretrained()` and load the saved model using `TFAutoModelForSequenceClassification.from_pretrained()`.

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("./sentiment_transfer_learning_tensorflow/")

# Load model
loaded_model = TFAutoModelForSequenceClassification.from_pretrained('./sentiment_transfer_learning_tensorflow/')

Some layers from the model checkpoint at ./sentiment_transfer_learning_tensorflow/ were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./sentiment_transfer_learning_tensorflow/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


To verify that the customized transfer learning model is loaded correctly, the loaded model is used to make predictions on the testing dataset. We can see that the prediction results are exactly the same as the fine-tuned model, confirming that the model is loaded correctly.

In [ ]:
# Predict logit using the loaded model
y_test_predict = loaded_model.predict(dict(tokenized_data_test))['logits']

# Take a look at the first 5 predictions
y_test_predict[:5]

67/67 [==============================] - 12s 134ms/step


array([[-2.1745813,  4.9053063, -3.2688015],
       [-4.33564  , -1.0755202,  5.0015435],
       [-5.835117 ,  1.378774 ,  3.7665915],
       [-3.865402 , -1.8417912,  5.4970236],
       [-2.2431595, -2.1613474,  4.6802683]], dtype=float32)